In [24]:
QUERY = "Is WEF the illumunati?"

In [25]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown


# Make sure to update the allowed IP addresses in the Google Custom Search Engine
load_dotenv()
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Point to the local server
# client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

client = OpenAI(base_url="https://api.openai.com/v1", api_key=os.getenv("OPEN_AI_KEY"))


In [26]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()


def top5_results(query):
    return search.results(query, 5)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

In [27]:
google_search_results = tool.run(QUERY)

In [28]:
google_search_results[0]["link"]

'https://www.weforum.org/agenda/2020/06/now-is-the-time-for-a-great-reset/'

In [29]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def fetch_page_text_with_uc(url):
    try:
        # Initialize the Chrome driver
        options = uc.ChromeOptions()
        
        # If you want Chrome to run headless (without opening a window)
        options.add_argument('--headless')
        
        driver = uc.Chrome(options=options)

        # Navigate to the page
        driver.get(url)
        
        # Wait for the page to load and ensure a certain element is present
        # Adjust the below line to wait for a specific element that indicates the page has loaded
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        
        # Now that the page is loaded, you can extract its text
        page_text = driver.find_element(By.TAG_NAME, "body").text
        
        return page_text
        
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Make sure to quit the driver to free up resources
        driver.quit()

In [30]:
completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "As the WEF_and_Global_Shapers_Expert, my role is to provide precise information on the World Economic Forum (WEF) and the Global Shapers Community. I synthesize data on WEF's missions and the impactful projects of Global Shapers, ensuring users receive the latest and most relevant insights. To achieve this, I will initially use text from official websites for responses. If the available text does not sufficiently answer a question, I will rely on my own knowledge, ensuring accuracy by only providing information I am 100% certain of. In cases where I cannot confirm the accuracy of my knowledge or the relevance of website content, I will honestly report having no information on the topic."},
    {"role": "user", "content": QUERY},
    {"role": "system", "content": "Website Body:" + fetch_page_text_with_uc(google_search_results[0]["link"])},
  ],
  temperature=0.7,
)

display(Markdown(completion.choices[0].message.content))

The World Economic Forum (WEF) is not related to the Illuminati or any similar conspiracy theories. The WEF is an international organization for public-private cooperation. Founded in 1971 by Klaus Schwab, it engages the foremost political, business, cultural, and other leaders of society to shape global, regional, and industry agendas. The organization is well-known for its Annual Meeting in Davos, Switzerland, where leaders from around the world gather to discuss and collaborate on international issues, including economic growth, environmental sustainability, and the advancement of technology.

The WEF focuses on initiatives that support sustainable development, stakeholder capitalism, and the harnessing of technology for the public good. It aims to address global challenges by fostering cooperation between the public and private sectors, promoting a multi-stakeholder approach to global governance. Any suggestions or theories linking the WEF to secret societies or conspiratorial organizations are unfounded and do not align with the openly stated goals and activities of the World Economic Forum.